In [ ]:
# Import Dependencies
import requests
import json
import time
import pandas as pd
from pandas import to_datetime
import numpy as np


# SQLAlchemy
from sqlalchemy import create_engine



In [ ]:
# Read the CSV file and put it into a dataframe
csv_file = 'Resources/original_book_data.csv'
book_df = pd.read_csv(csv_file, na_filter = True, na_values = '[]')
book_df

In [ ]:
# Filling NA values with other values to avoid data loss.
book_df['price'] = book_df['price'].fillna(0)
book_df['awards'] = book_df['awards'].fillna("['not awarded']")
book_df

In [ ]:
# Store the columns we want in a new DF
book_df2 = book_df[['title', 'author','rating', 'description', 'language', 'isbn',
    'genres', 'pages', 'publishDate', 'publisher', 'numRatings', 'likedPercent','awards','price']].copy()

# Filter DF to only include books in English and drop null values
book_df2 = book_df2[book_df2['language'] == 'English']
book_df2.dropna(inplace = True)
book_df2

In [ ]:
# Determine the column dtypes
book_df2.dtypes


In [ ]:
## convert the Title, Author, Language column to a string
book_df2['title'] = book_df2['title'].astype('string')
book_df2['author'] = book_df2['author'].astype('string')
book_df2['language'] = book_df2['language'].astype('string')
book_df2.dtypes

In [ ]:
## convert the ISBN column to a string
book_df2['isbn'] = book_df2['isbn'].astype('string')

# drop books with the ISBN of 9999999999999
book_df2.drop(book_df2[book_df2['isbn'] == '9999999999999'].index, inplace = True)

# remove duplicate ISBN numbers
book_df2.drop_duplicates(subset=['isbn'], keep = 'first', inplace = True)

book_df2

In [ ]:
test_df = book_df2[['author', 'isbn']].copy()
test_df = test_df.set_index('isbn')
test_df.head(12)

In [ ]:
# Split the Author Strings by a comma

new_df = pd.DataFrame(test_df['author'].str.split(pat=',', n=-1, expand=True))
new_df

In [ ]:
# Keep only the index and column of the first author
author_table = new_df.loc[:, :0]

#name the author column 
author_table.columns = ['author']

author_table

In [ ]:
# Keep only the authors name
split_author = pd.DataFrame(author_table['author'].str.split(pat='(', n=-1, expand=True))
split_author

In [ ]:
# Keep only the index and column of the first author
author_df = split_author.loc[:, :0]

#name the author column 
author_df.columns = ['main_author']

#rename the index
author_df = author_df.rename(index = {'' : 'isbn'})

author_df

In [ ]:
# merge data from the author table to the main data
main_book_data = pd.merge(book_df2, author_df, on = 'isbn')

#remove the original author column
main_book_data = main_book_data.drop(columns=['author'])

main_book_data

In [ ]:
# Filtering dates that doesn't follow correct format
data_df1 = main_book_data.loc[main_book_data["publishDate"].str.len()>8]
#data_df1.drop(columns="Unnamed: 0")
data_df1

In [ ]:
# Cleaned the published date column to bring all dates to one format
data_df1["publishedDate"]= pd.to_datetime(data_df1['publishDate'],errors = 'coerce')

# Drop all NA values 
data_df1=data_df1.dropna()
data_df1.head()

In [ ]:
# Creates new column with year published
data_df1['yearPublished'] = pd.DatetimeIndex(data_df1['publishedDate']).year
data_df1

In [ ]:
#Filtering the data for latest 5 yrs
data_df1=data_df1[(data_df1['yearPublished']>=2015)&(data_df1['yearPublished']<=2020)].reset_index(inplace=False)
data_df1

In [ ]:
# Making a copy of dataframe
awards_df = data_df1.copy()
awards_df

In [ ]:
#keep only the ISBN and awards Column
awards_df1 = awards_df[['isbn', 'awards']].copy()

awards_df1.head()

In [ ]:
# eliminate the [ ' ] marks in the awards column
awards_df1['awards'] = awards_df1['awards'].str.replace('[', '')
awards_df1['awards'] = awards_df1['awards'].str.replace(']', '')
awards_df1['awards'] = awards_df1['awards'].str.replace("'", "")
awards_df1['awards'] = awards_df1['awards'].str.replace('"', "")
awards_df1['awards'] = awards_df1['awards'].str.replace('()', "")
awards_df1


In [ ]:
#create a new df and flatten the awards into a new list
new_df_awards = awards_df1['awards'].copy()

genre_values = np.core.defchararray.split(new_df_awards.values.astype('str'),', ')
flatten_list_awards = [item for sublist in genre_values for item in sublist]
len(flatten_list_awards)

In [ ]:
#take the award list and add unique values to the unique_award_list
unique_award_list = []
for x in flatten_list_awards:
    if x not in unique_award_list:
        unique_award_list.append(x)

len(unique_award_list)

In [ ]:
#create a table of the unique award values
award_tables = pd.DataFrame(unique_award_list, columns = ['award'])

# Remove year in brackets
award_tables=award_tables['award'].str.split(pat='(', n=1,expand=True)
award_tables = award_tables.rename(columns={0:"award"})
award_unique = award_tables["award"].unique()

#create a table of the unique award values after removing year which in brackets
award_tables1 = pd.DataFrame(award_unique, columns = ['award'])
award_tables1

# Add a Primary Key to each award
award_tables1['award_id'] = award_tables1.index + 600
award_tables1

In [ ]:
# create a list of ISBN and awards that will be zipped into a df
last_isbn_list = []
last_award_list = []

for index, row in awards_df1.iterrows():
    row['awards'].split(',')
    row['awards'].split(' ')
    isbn = row['isbn']
    for x in row['awards'].split(','):
        last_isbn_list.append(isbn)
        last_award_list.append(x)

#Check the length of each

print(len(last_isbn_list))
print(len(last_award_list))

In [ ]:
# zip the isbn and awards into a df
book_award_table = pd.DataFrame(zip(last_isbn_list, last_award_list), columns = ['isbn', 'award'])

# strip leading and trailing white space in the awards column
book_award_table['award'] = book_award_table['award'].str.strip()

# Remove year in brackets
book_award_table['award']= book_award_table['award'].str.split(pat='(', n=1,expand=True)

book_award_table

In [ ]:
# merging the dataframes
merge_award_df = pd.merge(award_tables1,book_award_table,on="award")
merge_award_df = merge_award_df.drop(columns="award")
merge_award_df

In [ ]:
#keep only the ISBN and Genre Column
genre_df = data_df1[['isbn', 'genres']].copy()

genre_df.head()

In [ ]:
# eliminate the [ ' ] marks in the genres column

genre_df2 = genre_df.copy()
genre_df2['genres'] = genre_df2['genres'].str.replace('[', '')
genre_df2['genres'] = genre_df2['genres'].str.replace(']', '')
genre_df2['genres'] = genre_df2['genres'].str.replace("'", "")
genre_df2

In [ ]:
#create a new df and flatten the genres into a new list
new_df = genre_df2['genres'].copy()
genre_values = np.core.defchararray.split(new_df.values.astype('str'),', ')
flatten_list = [item for sublist in genre_values for item in sublist]
flatten_list

In [ ]:
#take the genres list and add unique values to the unique_genre_list
unique_genre_list = []
for x in flatten_list:
    if x not in unique_genre_list:
        unique_genre_list.append(x)

unique_genre_list

In [ ]:
# check the lengths of the two lists to make sure it worked 
print(len(flatten_list))
print(len(unique_genre_list))

In [ ]:
#create a table of the unique genre values
genre_tables = pd.DataFrame(unique_genre_list, columns = ['genre'])

# Add a Primary Key to each genre
genre_tables['genre_id'] = genre_tables.index + 500
genre_tables = genre_tables.loc[:,['genre_id','genre']]
genre_tables

In [ ]:
# create a list of ISBN and Genres that will be zipped into a df
last_isbn_list = []
last_genre_list = []

for index, row in genre_df2.iterrows():
    row['genres'].split(',')
    row['genres'].split(' ')
    isbn = row['isbn']
    for x in row['genres'].split(','):
        last_isbn_list.append(isbn)
        last_genre_list.append(x)

#Check the length of each

print(len(last_isbn_list))
print(len(last_genre_list))

In [ ]:
# zip the isbn and genres into a df
book_genre_table = pd.DataFrame(zip(last_isbn_list, last_genre_list), columns = ['isbn', 'genre'])

# strip leading and trailing white space in the genre column
book_genre_table['genre'] = book_genre_table['genre'].str.strip()

book_genre_table

In [ ]:
merge_genre_df = pd.merge(genre_tables,book_genre_table,on="genre")
merge_genre_df.drop(columns="genre")

In [ ]:
final_book_df = data_df1.drop(columns=["index","genres","publishDate","awards"])
final_book_df = final_book_df.loc[:,['isbn','title','language','main_author','description','pages','publisher','publishedDate','yearPublished','rating','numRatings','likedPercent','price']]
final_book_df

In [ ]:
final_book_df['price'] = final_book_df['price'].astype('float')
final_book_df['pages'] = final_book_df['pages'].astype('int')
final_book_df['publishedDate'] = final_book_df['publishedDate'].astype('string')
final_book_df.dtypes

In [ ]:
final_book_df.to_csv('cleaned_tables/final_book.csv', index=False)

In [ ]:
# author dataframe
author_info_df = pd.read_csv('scraped_author_data/birth_date1.csv')
author_info_df = author_info_df.rename(columns={"Unnamed: 0":"main_author","0":"dob"})
# remove duplicate ISBN numbers
author_info_df.drop_duplicates(subset=['main_author'], keep = 'first', inplace = True)
# author_info_df
author_info_df

# **Load**

In [ ]:
# connect to local database

protocol = 'postgresql'
username = 'postgres'
password = 'sairam'
host = 'localhost'
port = 5432
database_name = 'Book_db'
connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection_string)

In [ ]:
#preview table names

engine.table_names()

In [ ]:
### Use pandas to load DataFrame into database
author_info_df.to_sql('author', con=engine, if_exists='append', index=False)
award_tables1.to_sql('awards', con=engine, if_exists='append', index=False)
final_book_df.to_sql('book', con=engine, if_exists='append', index=False)
genre_tables.to_sql('genres', con=engine, if_exists='append', index=False)
merge_award_df.to_sql('award_isbn', con=engine, if_exists='append', index=False)
merge_genre_df.to_sql('genre_isbn', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from book', con=engine).head()

In [ ]:
pd.read_sql_query('select * from awards', con=engine).head()

In [ ]:
pd.read_sql_query('select * from genres', con=engine).head()

In [ ]:
pd.read_sql_query('select * from genre_isbn', con=engine).head()

In [ ]:
pd.read_sql_query('select * from award_isbn', con=engine).head()

In [ ]:
pd.read_sql_query('select * from author', con=engine).head()